# Simple notebook to demonstrate Sentiment analysis for Chatbox
This is a basic implementation of Sentiment Analysis with Keras model

In [0]:
!kill -9 -1

#### After wrong output during tflite convertion we load tensorflow nightly

In [1]:
!pip uninstall tensorflow -y
!pip install tf-nightly

Uninstalling tensorflow-2.2.0rc4:
  Successfully uninstalled tensorflow-2.2.0rc4
     |████████████████████████████████| 520.8MB 32kB/s 
     |████████████████████████████████| 2.9MB 53.7MB/s 
     |████████████████████████████████| 460kB 49.5MB/s 


#### Check versions of tensorflow and Keras

In [2]:
import tensorflow as tf
import tensorflow.keras as K

print(tf.__version__)
print(K.__version__)

2.2.0-dev20200505
2.2.4-tf


In [0]:
#Connect with drive

from google.colab import drive
drive.mount('/content/drive')

In [0]:
# read data from text files
with open('drive/My Drive/Smart Reply/10000_comments.csv', 'r') as f:
    reviews = f.read()
with open('drive/My Drive/Smart Reply/10000_labels.csv', 'r') as f:
    labels = f.read()

In [5]:
print(reviews[:1000])
print()
print(labels[:20])

﻿ξέρω ότι άκουσα νωρίτερα την κακή συνήθεια και άρχισα να φρικίνω από την πλευρά του = [
Ξαπλώστε στο κρεβάτι με πονοκέφαλο ... περιμένετε στην κλήση σας ...
Τελετή κηδείας ... ζοφερή Παρασκευή ...
θέλει να παρέα με φίλους Γρήγορα!
"Θέλουμε να ανταλλάξουμε με κάποιον που έχει εισιτήρια στο Χιούστον, αλλά κανείς δεν θα το κάνει."
- : Γιατί δεν πήγατε στο χορό; Π.Χ. το  μου δεν άρεσε στους φίλους μου
"Πρέπει να κοιμάμαι, αλλά όχι! να σκεφτώ έναν παλιό φίλο που θέλω. αλλά είναι παντρεμένος τώρα. γαμώτο, & ; με θέλει ! σκανδαλώδης!"
Χμμμ. Το ://../ είναι εκτός λειτουργίας
 αγάπη μου. Μου λείπεις
Λυπάμαι τουλάχιστον είναι Παρασκευή;
δεν μπορώ να κοιμηθώ
Πνιγμένος στους συγκρατητές της
Αχ! Πρέπει να νικήσω αυτό το ηλίθιο τραγούδι για να φτάσω στον επόμενο αγενή!
"αν παρακολουθήσετε τους λόφους στο Λονδίνο, θα συνειδητοποιήσετε τι είναι η περιοδεία γιατί ήταν εβδομάδες και εβδομάδες αργά, απλά παρακολουθώ το "
Πήρα τα νέα
Η καταιγίδα είναι εδώ και η ηλεκτρική ενέργεια έχει φύγει
Ο συμφώνησε
Έ

In [6]:
from string import punctuation

# get rid of punctuation
reviews = reviews.lower() # lowercase, standardize
all_text = ''.join([c for c in reviews if c not in punctuation])
print(all_text[:1000])

# split by new lines and spaces
reviews_split = all_text.split('\n')
print(reviews_split[:4])
print(len(reviews_split))
all_text = ' '.join(reviews_split)
print(all_text[:2000])

# create a list of words
words = all_text.split()
words[:10]

﻿ξέρω ότι άκουσα νωρίτερα την κακή συνήθεια και άρχισα να φρικίνω από την πλευρά του  
ξαπλώστε στο κρεβάτι με πονοκέφαλο  περιμένετε στην κλήση σας 
τελετή κηδείας  ζοφερή παρασκευή 
θέλει να παρέα με φίλους γρήγορα
θέλουμε να ανταλλάξουμε με κάποιον που έχει εισιτήρια στο χιούστον αλλά κανείς δεν θα το κάνει
  γιατί δεν πήγατε στο χορό πχ το  μου δεν άρεσε στους φίλους μου
πρέπει να κοιμάμαι αλλά όχι να σκεφτώ έναν παλιό φίλο που θέλω αλλά είναι παντρεμένος τώρα γαμώτο   με θέλει  σκανδαλώδης
χμμμ το  είναι εκτός λειτουργίας
 αγάπη μου μου λείπεις
λυπάμαι τουλάχιστον είναι παρασκευή
δεν μπορώ να κοιμηθώ
πνιγμένος στους συγκρατητές της
αχ πρέπει να νικήσω αυτό το ηλίθιο τραγούδι για να φτάσω στον επόμενο αγενή
αν παρακολουθήσετε τους λόφους στο λονδίνο θα συνειδητοποιήσετε τι είναι η περιοδεία γιατί ήταν εβδομάδες και εβδομάδες αργά απλά παρακολουθώ το 
πήρα τα νέα
η καταιγίδα είναι εδώ και η ηλεκτρική ενέργεια έχει φύγει
ο συμφώνησε
έτσι ξυπνήστε ξανά και δεν είναι καθόλου αργά αποτυ

['\ufeffξέρω',
 'ότι',
 'άκουσα',
 'νωρίτερα',
 'την',
 'κακή',
 'συνήθεια',
 'και',
 'άρχισα',
 'να']

In [7]:
# feel free to use this import 
from collections import Counter

## Build a dictionary that maps words to integers
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
print(len(vocab))
print(type(vocab))
#print first 10 words in vocab.These words are mostly encountered inside vocab
print(vocab[:10])
#Creating vocab to int dictionary
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
print(vocab_to_int)

##Creating all vocab to int
vocab_list_to_int = [vocab_to_int[word] for word in vocab]
print(vocab_list_to_int[:10])
print(len(vocab_list_to_int))

## use the dict to tokenize each review in reviews_split
## store the tokenized reviews in reviews_ints
reviews_ints = []
for review in reviews_split:
    reviews_ints.append([vocab_to_int[word] for word in review.split()])
    
#print first 2 reviews that have been changed to ints
print(reviews_ints[:2])

12278
<class 'list'>
['να', 'το', 'δεν', 'μου', 'είναι', 'και', 'για', 'στο', 'θα', 'με']
{'να': 1, 'το': 2, 'δεν': 3, 'μου': 4, 'είναι': 5, 'και': 6, 'για': 7, 'στο': 8, 'θα': 9, 'με': 10, 'από': 11, 'που': 12, 'ότι': 13, 'η': 14, 'αλλά': 15, 'είμαι': 16, 'την': 17, 'τα': 18, 'ο': 19, 'ένα': 20, 'πολύ': 21, 'σε': 22, 'αυτό': 23, 'του': 24, 'τον': 25, 'τόσο': 26, 'μια': 27, 'έχω': 28, 'πρέπει': 29, 'τη': 30, 'τώρα': 31, 'μπορώ': 32, 'σήμερα': 33, 'ήταν': 34, 'της': 35, 'σας': 36, 'γιατί': 37, 'θέλω': 38, 'οι': 39, 'στη': 40, 'δουλειά': 41, 'όχι': 42, 'μόλις': 43, 'στην': 44, 'τι': 45, 'τις': 46, 'έχει': 47, 'κάνω': 48, 'σπίτι': 49, 'μέρα': 50, 'ακόμα': 51, 'τους': 52, 'ναι': 53, 'πραγματικά': 54, 'όταν': 55, 'εδώ': 56, 'ξέρω': 57, 'αν': 58, 'μόνο': 59, 'πάω': 60, 'λείπει': 61, 'μην': 62, 'κάνει': 63, 'νομίζω': 64, 'ω': 65, 'στις': 66, 'ή': 67, 'νιώθω': 68, 'καλά': 69, 'ήμουν': 70, 'χωρίς': 71, 'μέχρι': 72, 'στον': 73, 'αύριο': 74, 'απόψε': 75, 'σαν': 76, 'ποτέ': 77, 'μισώ': 78, 'εκεί':

Use below code to generate .json file to use it inside android application

In [0]:
import json

with open('result.json', 'w', encoding='utf8') as fp:
   json.dump(vocab_to_int, fp, ensure_ascii=False)

In [8]:
# stats about vocabulary
print('Unique words:', len(vocab_to_int))  # should ~ 12000+
print()

# print tokens in first reviews
print('Tokenized review: \n', reviews_ints[:2])

Unique words: 12278

Tokenized review: 
 [[5572, 13, 464, 448, 17, 357, 5573, 6, 2206, 1, 5574, 11, 17, 1425, 24], [1853, 8, 200, 10, 252, 522, 44, 756, 36]]


Transform labels from 1-3

In [9]:
import numpy as np
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
print(labels_split)
print(len(labels_split))
encoded_labels = []


for i in labels_split:
  if i == 'sadness':
    encoded_labels.append(0)
  elif i == 'enthusiasm':
    encoded_labels.append(1)
  elif i == 'neutral':
    encoded_labels.append(2)
    
print(encoded_labels)
print(len(encoded_labels))

transformed_encoded_labels = np.array(encoded_labels)
print(transformed_encoded_labels)
#encoded_labels = np.array([1 if label == 'empty' 2 if label == 'sadness' else 0 for label in labels_split])
#print(encoded_labels)

['sadness', 'sadness', 'sadness', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'neutral', 'sadness', 'sadness', 'neutral', 'neutral', 'sadness', 'sadness', 'enthusiasm', 'sadness', 'enthusiasm', 'sadness', 'neutral', 'sadness', 'neutral', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'sadness', 'neutral', 'neutral', 'neutral', 'enthusiasm', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'neutral', 'neutral', 'sadness', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'neutral', 'sadness', 'sadness', 'sadness', 'sadness', 'neutral', 'neutral', 'sadness', 'neutral', 'enthusiasm', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'sadness', 'neutral', 'neutral', 'neu

## Tokenize labels to 1-13

In [0]:
import numpy as np
# 1=positive, 0=negative label conversion
labels_split = labels.split('\n')
print(labels_split)
print(len(labels_split))
encoded_labels = []


for i in labels_split:
  if i == 'empty':
    encoded_labels.append(0)
  elif i == 'sadness':
    encoded_labels.append(1)
  elif i == 'enthusiasm':
    encoded_labels.append(2)
  elif i == 'neutral':
    encoded_labels.append(3)
  elif i == 'worry':
    encoded_labels.append(4)
  elif i == 'love':
    encoded_labels.append(5)
  elif i == 'fun':
    encoded_labels.append(6)
  elif i == 'surprise':
    encoded_labels.append(7)
  elif i == 'hate':
    encoded_labels.append(8)
  elif i == 'happiness':
    encoded_labels.append(9)
  elif i == 'boredom':
    encoded_labels.append(10)
  elif i == 'relief':
    encoded_labels.append(11)
  elif i == 'anger':
    encoded_labels.append(12)
    
print(encoded_labels)
print(len(encoded_labels))

transformed_encoded_labels = np.array(encoded_labels)
print(transformed_encoded_labels)
#encoded_labels = np.array([1 if label == 'empty' 2 if label == 'sadness' else 0 for label in labels_split])
#print(encoded_labels)

['empty', 'sadness', 'sadness', 'enthusiasm', 'neutral', 'worry', 'sadness', 'worry', 'sadness', 'sadness', 'neutral', 'worry', 'sadness', 'sadness', 'surprise', 'sadness', 'love', 'sadness', 'worry', 'sadness', 'worry', 'fun', 'neutral', 'worry', 'sadness', 'worry', 'sadness', 'worry', 'sadness', 'worry', 'hate', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'sadness', 'worry', 'neutral', 'neutral', 'happiness', 'fun', 'worry', 'worry', 'empty', 'worry', 'worry', 'sadness', 'neutral', 'neutral', 'neutral', 'worry', 'empty', 'neutral', 'neutral', 'worry', 'enthusiasm', 'neutral', 'neutral', 'sadness', 'worry', 'sadness', 'sadness', 'sadness', 'sadness', 'neutral', 'worry', 'sadness', 'worry', 'happiness', 'neutral', 'worry', 'neutral', 'neutral', 'worry', 'neutral', 'neutral', 'happiness', 'worry', 'worry', 'sadness', 'neutral', 'sadness', 'worry', 'sadness', 'worry', 'hate', 'worry', 'worry', 'worry', 'neutral', 'neutral', 'worry', 'neutral', 'love', 'sadness', 'sadness', 'sa

In [10]:
# outlier review stats
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 40
Maximum review length: 34


In [11]:
print('Number of reviews before removing outliers: ', len(reviews_ints))

## remove any reviews/labels with zero length from the reviews_ints list.

# get indices of any reviews with length 0
non_zero_idx = [ii for ii, review in enumerate(reviews_ints) if len(review) != 0]

# remove 0-length reviews and their labels
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
transformed_encoded_labels = np.array([transformed_encoded_labels[ii] for ii in non_zero_idx])

print('Number of reviews after removing outliers: ', len(reviews_ints))
print('Number of labels after removing outliers: ', len(transformed_encoded_labels))


Number of reviews before removing outliers:  9976
Number of reviews after removing outliers:  9936
Number of labels after removing outliers:  9936


---
## Padding sequences

To deal with both short and very long reviews, we'll pad or truncate all our reviews to a specific length. For reviews shorter than some `seq_length`, we'll pad with 0s. For reviews longer than `seq_length`, we can truncate them to the first `seq_length` words. A good `seq_length`, in this case, is 40.

> **Exercise:** Define a function that returns an array `features` that contains the padded data, of a standard size, that we'll pass to the network. 
* The data should come from `review_ints`, since we want to feed integers to the network. 
* Each row should be `seq_length` elements long. 
* For reviews shorter than `seq_length` words, **left pad** with 0s. That is, if the review is `['best', 'movie', 'ever']`, `[117, 18, 128]` as integers, the row will look like `[0, 0, 0, ..., 0, 117, 18, 128]`. 
* For reviews longer than `seq_length`, use only the first `seq_length` words as the feature vector.

As a small example, if the `seq_length=10` and an input review is: 
```
[117, 18, 128]
```
The resultant, padded sequence should be: 

```
[0, 0, 0, 0, 0, 0, 0, 117, 18, 128]
```

**Your final `features` array should be a 2D array, with as many rows as there are reviews, and as many columns as the specified `seq_length`.**

This isn't trivial and there are a bunch of ways to do this. But, if you're going to be building your own deep learning networks, you're going to have to get used to preparing your data.

In [0]:
def pad_features(reviews_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    
    ##RANDOM GENERATION OF FEATURES TO PASS INTO NETWORK, 1 row seq_length columns
    ##genarates 1 row 100 columns with numbers from 1 to 12200+ as the length of vocabulary
    #features_random = np.random.randn(1, seq_length)
    #features_random = np.random.choice(vocab_list_to_int,(1,seq_length))
    #print(features_random)
    #print(features_random.shape)  
    
    #print(len(features))

    # for each review, I grab that review and 
    for i, row in enumerate(reviews_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [13]:
# Test your implementation!

seq_length = 40

features = pad_features(reviews_ints, seq_length=seq_length)

## test statements - do not change - ##
assert len(features)==len(reviews_ints), "Your features should have as many rows as reviews."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:2,:200])
print(type(features))
print(features.shape)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0 5572   13  464
   448   17  357 5573    6 2206    1 5574   11   17 1425   24]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 1853    8  200   10  252  522   44  756   36]]
<class 'numpy.ndarray'>
(9936, 40)


In [14]:
from keras.utils import to_categorical
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)

## split data into training, validation, and test data (features and labels, x and y)
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = transformed_encoded_labels[:split_idx], transformed_encoded_labels[split_idx:]

print(train_x.shape)
print(train_y.shape)
print(transformed_encoded_labels.shape)
print(transformed_encoded_labels)

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

#Now we can convert both the training and test labels into one-hot encoded vectors:
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
val_y = to_categorical(val_y)

print()
print(train_y)
print(train_y.shape)

(7948, 40)
(7948,)
(9936,)
[0 0 0 ... 2 1 1]
			Feature Shapes:
Train set: 		(7948, 40) 
Validation set: 	(994, 40) 
Test set: 		(994, 40)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]
(7948, 3)


Using TensorFlow backend.


## Build a Keras model to perform training

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, RNN, SimpleRNN, GRU

embedding_size=20
vocabulary_size = len(vocab_to_int)

model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=seq_length))
model.add(LSTM(64))
model.add(Dense(3, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 20)            245560    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                21760     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 267,515
Trainable params: 267,515
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
#Use categorical cross entropy as we have 3 labels
model.compile(loss='categorical_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint 
import datetime 

batch_size = 40
num_epochs = 5

checkpoint = ModelCheckpoint(filepath='sentiment_keras_LSTM.weights.best.hdf5', verbose=1, 
                               save_best_only=False)

!mkdir -p logs/fit/
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

callbacks_list = [checkpoint,tensorboard_callback]

model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list)
 

Epoch 1/5
  1/199 [..............................] - ETA: 0s - loss: 1.0974 - accuracy: 0.4500WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
194/199 [============================>.] - ETA: 0s - loss: 1.0146 - accuracy: 0.5174
Epoch 00001: saving model to sentiment_keras_LSTM.weights.best.hdf5
199/199 [==============================] - 2s 10ms/step - loss: 1.0131 - accuracy: 0.5190 - val_loss: 1.0099 - val_accuracy: 0.5141
Epoch 2/5
193/199 [============================>.] - ETA: 0s - loss: 0.9668 - accuracy: 0.5324
Epoch 00002: saving model to sentiment_keras_LSTM.weights.best.hdf5
199/199 [==============================] - 2s 8ms/step - loss: 0.9647 - accuracy: 0.5340 - val_loss: 1.0181 - val_accuracy: 0.5181
Epoch 3/5
198/199 [============================>.

## If you come later you can load the model

In [0]:
#new_model = tf.keras.models.load_model('sentiment_keras_LSTM.weights.best.hdf5')

In [18]:
!mkdir -p saved_model
new_model = model
new_model.save('saved_model/my_model') 

#new_model.compute_output_shape

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [20]:
#print(new_model.get_input_shape_at(0))
print(new_model)
for node in model.outputs:
  print(node)

Tensor("dense/Identity:0", shape=(None, 3), dtype=float32)


# Convert to tflite for android 

In [21]:
tflite_model = tf.keras.models.load_model('saved_model/my_model')
converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
tflite_save = converter.convert()
open("greek_smart_reply_model.tflite", "wb").write(tflite_save)

1083272

In [22]:
interpreter = tf.lite.Interpreter('greek_smart_reply_model.tflite')
#interpreter.allocate_tensors()

#input_details = interpreter.get_input_details()
print(interpreter.get_output_details())
interpreter.get_tensor_details()

[{'name': 'Identity', 'index': 38, 'shape': array([1, 3], dtype=int32), 'shape_signature': array([-1,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


[{'dtype': numpy.float32,
  'index': 0,
  'name': 'embedding_input',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 40], dtype=int32),
  'shape_signature': array([-1, 40], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 1,
  'name': 'sequential/dense/BiasAdd/ReadVariableOp/resource',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([3], dtype=int32),
  'shape_signature': array([3], dtype=int32),
  'sparsity_parameters': {}},
 {'dtype': numpy.float32,
  'index': 2,
  'name': 'sequential/embedding/embedding_lookup/20519',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},


In [0]:
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 20)            245560    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                21760     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 267,515
Trainable params: 267,515
Non-trainable params: 0
_________________________________________________________________


In [42]:
# zip saved model folder
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/my_model/ (stored 0%)
  adding: content/saved_model/my_model/variables/ (stored 0%)
  adding: content/saved_model/my_model/variables/variables.data-00000-of-00002 (deflated 13%)
  adding: content/saved_model/my_model/variables/variables.index (deflated 61%)
  adding: content/saved_model/my_model/variables/variables.data-00001-of-00002 (deflated 6%)
  adding: content/saved_model/my_model/saved_model.pb (deflated 90%)
  adding: content/saved_model/my_model/assets/ (stored 0%)


## Test your model with test dataset

In [23]:
scores = new_model.evaluate(test_x, test_y, verbose=1)

16/32 [==============>...............] - ETA: 0s - loss: 1.5754 - accuracy: 0.4473

InvalidArgumentError: ignored

## Test a review

In [0]:
# negative test review
test_review_neg = 'ξαπλώστε στο κρεβάτι με πονοκέφαλο  περιμένετε στην κλήση σας'
test_review_pos = 'θέλει να παρέα με φίλους γρήγορα'
test_review_neutral = 'γιατί δεν πήγατε στο χορό πχ το  μου δεν άρεσε στους φίλους μου'


In [25]:
from string import punctuation

def tokenize_review(test_review):
    test_review = test_review.lower() # lowercase
    # get rid of punctuation
    test_text = ''.join([c for c in test_review if c not in punctuation])

    # splitting by spaces
    test_words = test_text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints

# test code and generate tokenized review
test_ints = tokenize_review(test_review_pos)
print(test_ints)

[[164, 1, 757, 10, 245, 501]]


In [26]:
# test sequence padding
seq_length=40
features = pad_features(test_ints, seq_length)

print(features)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 164   1
  757  10 245 501]]


In [27]:
# Use test set to find accuracy
predictions = np.round(new_model.predict(features))
print(predictions[0])

predictions = new_model.predict(features)
print(predictions[0])


[0. 1. 0.]
[0.22111712 0.74994016 0.02894274]


In [0]:
def predict(net, test_review, sequence_length=40):
     
    # tokenize review
    test_ints = tokenize_review(test_review)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    pred = np.round(net.predict(features))
    print(net.predict(features)[0])
    print(pred)
    # printing output value, before rounding    
    # print custom response
    
        

In [30]:
predict(new_model,test_review_neutral)

[0.53470016 0.01791311 0.44738674]
[[1. 0. 0.]]


In [31]:
test_review_atomic = 'δείτε το του συλλόγου στα γράμματα'
predict(new_model,test_review_pos)

[0.22111712 0.74994016 0.02894274]
[[0. 1. 0.]]


### Getting Tensorboard to see the graph

In [35]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [36]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-05-06 04:51:39.908137: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.6/dist-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 289, in main
    return runner(self.flags) or 0
  File "/usr/local/lib/python3.6/dist-packages/tensorboard/program.py", line 305, in _run_serve_subcommand
    server = self._make_server()
  File "/usr/local/lib/python3